# Use VeinsGym with the Straight Scenario

This notebook registers and runs the `straight_scenario` with VeinsGym. The agent observes features relevant for task offloading and returns an action: 0 = no offload, 1/2/3 = offload to RSU 0/1/2.

## Dependencies

You'll need SUMO, OMNeT++, a C++ compiler, ZeroMQ, and the `veins_gym` Python package.

In [1]:
import numpy as np
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_

In [2]:
import gym
import veins_gym

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
# Ensure OMNeT++ environment is available inside this kernel
import os, subprocess, shutil, sys, pathlib

def source_bash_env(script_path: str):
    cmd = f"bash -lc 'source {script_path} && env -0'"
    out = subprocess.check_output(cmd, shell=True)
    for chunk in out.split(b'\x00'):
        if not chunk: continue
        k, _, v = chunk.partition(b'=')
        if k: os.environ[k.decode()] = v.decode()

omnetpp_setenv = "/home/abhay/omnetpp-5.7.1/setenv"
if os.path.exists(omnetpp_setenv):
    source_bash_env(omnetpp_setenv)
    print('Sourced:', omnetpp_setenv)
else:
    raise FileNotFoundError(f'OMNeT++ setenv not found at {omnetpp_setenv}. Update the path if installed elsewhere.')

print('opp_run path:', shutil.which('opp_run'))
try:
    ver = subprocess.run(['opp_run', '-v'], check=False, capture_output=True, text=True).stdout.splitlines()[0]
    print(ver)
except Exception as e:
    print('Warning: opp_run not callable:', e)

# Preflight checks for straight scenario (now using ../scenario)
base = pathlib.Path.cwd()
sc_dir = (base / '..' / 'scenario').resolve()
print('scenario dir:', sc_dir)
print('omnetpp.ini exists:', (sc_dir / 'omnetpp.ini').exists())
print('run script exists:', (sc_dir / 'run').exists())
print('sumo path:', shutil.which('sumo'))

Sourced: /home/abhay/omnetpp-5.7.1/setenv
opp_run path: /home/abhay/omnetpp-5.7.1/bin/opp_run
OMNeT++ Discrete Event Simulation  (C) 1992-2021 Andras Varga, OpenSim Ltd.
scenario dir: /home/abhay/Major_Project/vanet_task_offloading/scenario
omnetpp.ini exists: True
run script exists: True
sumo path: /usr/bin/sumo


In [4]:
# Register the Straight scenario as a Gym environment (no extra plugin lib needed)
import os, pathlib, subprocess, shutil
base = pathlib.Path.cwd()
sc_dir = (base / '..' / 'scenario').resolve()
assert (sc_dir / 'omnetpp.ini').exists(), 'Missing scenario/omnetpp.ini'
gym.register(
    id='veins-straight-v1',
    entry_point='veins_gym:VeinsEnv',
    kwargs={
        'scenario_dir': '../scenario',
        'timeout': 7.0,
        'print_veins_stdout': True,
        # 'user_interface': 'Cmdenv',
        'config': 'StraightRoad',
        # "run_veins": False,  # do not start veins through Veins-Gym
		# "port": 5555, 
    },
)
print('Registered veins-straight-v1 with scenario_dir:', sc_dir)

Registered veins-straight-v1 with scenario_dir: /home/abhay/Major_Project/vanet_task_offloading/scenario


Create the environment and step it with random actions for a quick smoke test.

In [5]:
env = gym.make('veins-straight-v1')
obs = env.reset()
print('Initial observation shape:', np.array(obs).shape)
done = False
steps = 0
while not done:
    action = env.action_space.sample()  # 0..3
    obs, reward, done, info = env.step(action)
    steps += 1
print('Episode steps:', steps)

/home/abhay/anaconda3/envs/mp/lib/python3.13/site-packages/gym/utils/passive_env_checker.py:181: UserWarning: WARN: The default seed argument in `Env.reset` should be `None`, otherwise the environment will by default always be deterministic. Actual default: seed='NO SEED GIVEN'
  logger.warn(
/home/abhay/anaconda3/envs/mp/lib/python3.13/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
/home/abhay/anaconda3/envs/mp/lib/python3.13/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


Initial observation shape: (7,)
Episode steps: 830


## Example: Mean Reward

Roll out once and compute mean reward.

In [6]:
obs = env.reset()
done = False
rewards = []
while not done:
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    rewards.append(reward)
print('Number of steps:', len(rewards))
print('Mean reward:', float(np.mean(rewards)) if rewards else 0.0)

Number of steps: 830
Mean reward: 0.0
